In [1]:
from keras.models import Sequential , Model
from keras.layers import Dense ,  BatchNormalization , Reshape , Input , Flatten
from keras.layers import Conv2D , MaxPool2D , Conv2DTranspose , UpSampling2D , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU , PReLU
from keras.layers import Activation
from keras.layers import Dropout

from keras.layers import Concatenate

#addin cycleGAN 使用instance-norm
from keras_contrib.layers.normalization import InstanceNormalization

from keras.initializers import truncated_normal , constant , random_normal

from keras.optimizers import Adam , RMSprop

#残差块使用
from keras.layers import Add

from keras.datasets import mnist

#导入存在的模型
from keras.applications import VGG16 , VGG19

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os

import matplotlib as plt
import numpy as np

import gc

from glob import glob

import keras.backend as K

import scipy

%matplotlib inline

In [8]:
WIDTH = 256
HEIGHT = 256
CHANNEL = 3

SHAPE = (WIDTH , HEIGHT , CHANNEL)

BATCH_SIZE = 4 #crazy!!! slow turtle
EPOCHS = 10

PATH = '../dataset/vangogh2photo/'

#生成多少个图像 长*宽
ROW = 2 #几行决定显示几个测试样例 显示2个
COL = 3 #3列是因为要显示 原图像 另一个特征空间的图像 还原后的图像

TRAIN_APPLE_PATH = glob(PATH + 'trainA/*')
TRAIN_ORANGE_PATH = glob(PATH + 'trainB/*')
TEST_APPLE_PATH = glob(PATH + 'testA/*')
TEST_ORANGE_PATH = glob(PATH + 'testB/*')

#卷积使用 基卷积核大小
G_filters = 64
D_filters = 64


In [4]:
patch = int(HEIGHT/(2**4)) #16
disc_patch = (patch , patch , 1) #16*16*1



In [5]:
def load_image(batch_size = BATCH_SIZE , training = True):
    #随机在图片库中挑选
    if training:
        APPLE_PATH = TRAIN_APPLE_PATH
        ORANGE_PATH = TRAIN_ORANGE_PATH
    else:
        APPLE_PATH = TEST_APPLE_PATH
        ORANGE_PATH = TEST_ORANGE_PATH
        
    images_apple = np.random.choice(APPLE_PATH , size=batch_size)
    images_orange = np.random.choice(ORANGE_PATH , size=batch_size)
    
    apples = []
    oranges = []
    
    for i in range(batch_size):
        apple = scipy.misc.imread(images_apple[i] , mode='RGB').astype(np.float)
        orange = scipy.misc.imread(images_orange[i] , mode='RGB').astype(np.float)
        
        #随机性地对训练样本进行 左右反转
        if training and np.random.random()<0.5:
            apple = np.fliplr(apple)
            orange = np.fliplr(orange)
        
        apples.append(apple)
        oranges.append(orange)
        
    apples = np.array(apples)/127.5 - 1
    oranges = np.array(oranges)/127.5 - 1
    
    return apples , oranges


def write_image(epoch):
    #生成高分图像时 进行对比显示
    apples , oranges = load_image(batch_size=1 , training=False) #1个batch就是两幅图像 一个苹果的 一个橘子的
    
    fake_apples = generator_apple2orange.predict(apples) #橘子风格的苹果
    fake_oranges = generator_orange2apple.predict(oranges) #苹果风格的橘子
    
    apples_hat = generator_orange2apple.predict(fake_apples) #还原后的苹果
    oranges_hat = generator_apple2orange.predict(fake_oranges) #还原后的橘子
    
    
    apples = apples*0.5+0.5
    oranges = oranges*0.5+0.5
    
    fake_apples = fake_apples*0.5+0.5
    fake_oranges = fake_oranges*0.5+0.5
    
    apples_hat = apples_hat*0.5+0.5
    oranges_hat = oranges_hat*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    count=0
    
    axes[0][0].imshow(apples[0])
    axes[0][0].set_title('apple')
    axes[0][0].axis('off')

    axes[0][1].imshow(fake_apples[0])
    axes[0][1].set_title('apple-orange')
    axes[0][1].axis('off')
    
    axes[0][2].imshow(apples_hat[0])
    axes[0][2].set_title('restruct apple')
    axes[0][2].axis('off')

    axes[1][0].imshow(oranges[0])
    axes[1][0].set_title('orange')
    axes[1][0].axis('off')

    axes[1][1].imshow(fake_oranges[0])
    axes[1][1].set_title('orange-apple')
    axes[1][1].axis('off')
    
    axes[1][2].imshow(oranges_hat[0])
    axes[1][2].set_title('restruct orange')
    axes[1][2].axis('off')
      
    fig.savefig('edges2shoes_discogan/No.%d.png' % epoch)
    plt.pyplot.close()


In [6]:
#==============

In [6]:
def conv2d(input_data , output_size , filter_size=4 , instance_norm=True):
    h = Conv2D(output_size , filter_size , strides=(2,2) , padding='same')(input_data)
    h = LeakyReLU(alpha=0.2)(h)
    
    if instance_norm:
        h = InstanceNormalization()(h)
    
    return h


#实现U-Net使用 需要网络的跳连接
def deconv2d(input_data , skip_input , output_size , filter_size=4 , dropout_rate=0.0):
    h = UpSampling2D(size=2)(input_data)
    h = Conv2D(output_size , filter_size , strides=(1,1) , padding='same')(h)
    h = Activation('relu')(h)
    
    if dropout_rate:
        h = Dropout(rate=dropout_rate)(h)
    
    h = InstanceNormalization()(h)
    h =  Concatenate()([h , skip_input]) #跳连接具体实现

    return h
    

In [7]:
#G使用encoder-decoder结构 但是需要引入跳连接 即U-Net
def generator(G_filters , name):
    style = Input(shape=SHAPE) #输入一个风格的图像 生成另一个风格的图像
    
    #encoder
    d1 = conv2d(style , G_filters , instance_norm=False)
    d2 = conv2d(d1 , G_filters*2)
    d3 = conv2d(d2 , G_filters*4)
    d4 = conv2d(d3 , G_filters*8)
    d5 = conv2d(d4 , G_filters*8)
    d6 = conv2d(d5 , G_filters*8)
    d7 = conv2d(d6 , G_filters*8)
 

    #decoder
    
    u1 = deconv2d(d7 , d6 , G_filters*8)
    u2 = deconv2d(u1 , d5 , G_filters*8)
    u3 = deconv2d(u2 , d4 , G_filters*8)
    u4 = deconv2d(u3 , d3 , G_filters*4)
    u5 = deconv2d(u4 , d2 , G_filters*2)
    u6 = deconv2d(u5 , d1 , G_filters)
    
    u7 = UpSampling2D(size=(2,2))(u6)
    other_style = Conv2D(filters=CHANNEL , kernel_size=(4,4) , strides=(1,1) , padding='same' , activation='tanh')(u7) #还原后的图像
    
    return Model(style , other_style , name=name)

In [8]:
def discriminator(D_filters , name):
    style = Input(shape=SHAPE) #风格1 的图像
    #style2 = Input(shape=SHAPE) #风格2 的图像
    
    #style = Concatenate()([style1 , style2])
    
    h1 = conv2d(style , output_size=D_filters , instance_norm=False)
    h2 = conv2d(h1 , output_size=D_filters*2)
    h3 = conv2d(h2 , output_size=D_filters*4)
    h4 = conv2d(h3 , output_size=D_filters*8)
    
    validity =  Conv2D(1 , kernel_size=(4,4) , strides=(1,1) , padding='same')(h4)
    
    return Model(style , validity , name=name)

In [25]:
adam = Adam(lr = 0.0002 , beta_1=0.5)

discriminator_apple = discriminator(D_filters , name='discriminator_apple') #判别苹果风格
discriminator_apple.compile(optimizer = adam , loss='mse' , metrics=['accuracy'])
discriminator_orange = discriminator(D_filters , name='discriminator_orange') #判别橘子风格
discriminator_orange.compile(optimizer = adam , loss='mse' , metrics=['accuracy'])

generator_apple2orange = generator(G_filters , name='generator_apple2orange')
generator_orange2apple = generator(G_filters , name='generator_orange2apple')

apples = Input(shape=SHAPE)
oranges = Input(shape=SHAPE)

fake_apples = generator_apple2orange(apples) #使用G来 将苹果变成橘子风格的苹果
fake_oranges = generator_orange2apple(oranges) #使用F来 将橘子变成苹果风格的橘子

apples_hat = generator_orange2apple(fake_apples) #使用F将橘子风格的苹果还原为原苹果
oranges_hat = generator_apple2orange(fake_oranges) #使用G将苹果风格的橘子还原为原橘子

#freeze D
discriminator_apple.trainable = False
discriminator_orange.trainable = False

validity_apple = discriminator_apple(fake_oranges) #真苹果 和 苹果风格的橘子 之间的潜在模式相似度
validity_orange = discriminator_orange(fake_apples) #真橘子 和 橘子风格的苹果 之间的潜在模式相似度

#一共6个输出 最后4个输出希望和原图像一致 这样图像同时具有两个风格
combined = Model([apples , oranges] , [validity_apple , validity_orange , fake_apples , fake_oranges , apples_hat , oranges_hat])
combined.compile(optimizer=adam , loss=['mse' , 'mse' , 'mae' , 'mae' , 'mae' , 'mae']) # , loss_weights=[1 ,1,10, 10 , 1, 1]) #cycleGAN的损失权重

In [1]:
#tuple类型相加 相当于cat连接
real_labels = np.ones(shape=(BATCH_SIZE , )+disc_patch) 
fake_labels = np.zeros(shape=(BATCH_SIZE , )+disc_patch)


for i in range(1001):
    apples_ , oranges_ = load_image()
    
    fake_apples_ = generator_apple2orange.predict(apples_) #使用G将苹果变成 橘子风格的苹果
    fake_oranges_ = generator_orange2apple.predict(oranges_) #使用F将橘子变成 苹果风格的橘子
    #训练判别器
    apple_loss = discriminator_apple.train_on_batch(apples_ , real_labels)
    fake_apple_loss = discriminator_apple.train_on_batch(fake_apples_ , fake_labels)
    loss_apple = np.add(apple_loss , fake_apple_loss)/2

    orange_loss = discriminator_orange.train_on_batch(oranges_ , real_labels)
    fake_orange_loss = discriminator_orange.train_on_batch(fake_oranges_ , fake_labels)
    loss_orange = np.add(orange_loss , fake_orange_loss)/2

    loss = np.add(loss_apple , loss_orange)/2

    #训练生成器
    generator_loss = combined.train_on_batch([apples_ , oranges_] , [real_labels , real_labels , oranges_ , apples_ , apples_ , oranges_])
    
    print('epoch:%d loss:%f accu:%f |mse1:%f :mse2:%f mae1:%f mae2:%f mae3:%f mae4:%f' % (i , loss[0] , loss[1] , generator_loss[0] , generator_loss[1] , generator_loss[2] , generator_loss[3] , generator_loss[4] , generator_loss[5]))

    if i % 50 == 0:
        write_image(i)

write_image(999)


NameError: name 'np' is not defined

In [70]:
real_labels.shape

(64, 1)

In [ ]:
gc.collect()

In [ ]:
gc.collect()